In [605]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException

from tqdm import tqdm

import re
import time
import json

In [194]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [154]:
test_data = pd.read_csv('test.csv')

In [260]:
test_data.iloc[12561]['description']

'Машина в отличном состоянии!!17.10.2020 было сделано Т.О заменено масло в коробке передач в двигателе,так же заменены все фильтра.Почти максимальная комплектация кроме люка!!! Без ключевой доступ. В ДТП не была не билась.Красился капот и передний бампер так как от времени были сколы.Капот фары туманки затянуты броне пленкой.Делали для себя супруга не ездит вся причина продажи.. Новая летняя резина. Продаю не спеша. '

In [473]:
valid_data = pd.DataFrame()
catalog_data = pd.DataFrame()

In [464]:
valid_data

,year,bodytype,kmage,color,engineDisplacement,enginePower,fuelType,vehicleTransmission,drive,wheel,state,owner,pts,customs,owningTime,description,sell_id


In [197]:
test_data.iloc[1550]

bodyType                                                внедорожник 5 дв.
brand                                                               SKODA
car_url                 https://auto.ru/cars/used/sale/skoda/kodiaq/11...
color                                                               серый
complectation_dict      {"id":"21404914","name":"Hockey Edition","avai...
description             Куплена в январе 2019 г. у официала. Производс...
engineDisplacement                                                2.0 LTR
enginePower                                                       180 N12
equipment_dict          {"cruise-control":true,"tinted-glass":true,"es...
fuelType                                                           бензин
image                   https://autoru.naydex.net/ESooa9149/468b9eoMem...
mileage                                                             34000
modelDate                                                            2016
model_info              {"code":"KODIA

In [198]:
test_data.iloc[1550]['car_url']

'https://auto.ru/cars/used/sale/skoda/kodiaq/1101318069-a7be9a63/'

In [155]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34686 entries, 0 to 34685
Data columns (total 32 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   bodyType              34686 non-null  object
 1   brand                 34686 non-null  object
 2   car_url               34686 non-null  object
 3   color                 34686 non-null  object
 4   complectation_dict    6418 non-null   object
 5   description           34686 non-null  object
 6   engineDisplacement    34686 non-null  object
 7   enginePower           34686 non-null  object
 8   equipment_dict        24690 non-null  object
 9   fuelType              34686 non-null  object
 10  image                 34686 non-null  object
 11  mileage               34686 non-null  int64 
 12  modelDate             34686 non-null  int64 
 13  model_info            34686 non-null  object
 14  model_name            34686 non-null  object
 15  name                  34686 non-null

In [6]:
test_data['bodyType'].unique()

array(['лифтбек', 'внедорожник 5 дв.', 'хэтчбек 5 дв.', 'седан',
       'компактвэн', 'универсал 5 дв.', 'пикап одинарная кабина',
       'хэтчбек 3 дв.', 'купе', 'кабриолет', 'минивэн',
       'пикап двойная кабина', 'внедорожник 3 дв.', 'родстер', 'микровэн',
       'седан 2 дв.', 'купе-хардтоп', 'фастбек', 'тарга',
       'внедорожник открытый', 'лимузин', 'пикап полуторная кабина',
       'седан-хардтоп', 'фургон'], dtype=object)

In [606]:
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)



====== WebDriver manager ======
Could not get version for google-chrome with the command:  powershell "$ErrorActionPreference='silentlycontinue' ; (Get-Item -Path "$env:PROGRAMFILES\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion ; if (-not $? -or $? -match $error) { (Get-Item -Path "$env:PROGRAMFILES(x86)\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion } if (-not $? -or $? -match $error) { (Get-Item -Path "$env:LOCALAPPDATA\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion } if (-not $? -or $? -match $error) { reg query "HKCU\SOFTWARE\Google\Chrome\BLBeacon" /v version } if (-not $? -or $? -match $error) { reg query "HKLM\SOFTWARE\Wow6432Node\Microsoft\Windows\CurrentVersion\Uninstall\Google Chrome" /v version }"
Current google-chrome version is UNKNOWN
Get LATEST chromedriver version for UNKNOWN google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/99.0.4844.51/chromedriver_win32.zip
Driver has been s

In [514]:
regions = ['leningradskaya_oblast', 'moskovskaya_oblast']
models = ['skoda', 'mini']

In [445]:
urls_list = []

In [453]:
def get_urls_for_model_in_region(model, region):
    
    result_list = []
    cnt = True
    page_num = 1
    
    while cnt is True:
        try:
            driver.get(f'https://auto.ru/{region}/cars/{model}/used/?page={page_num}')
            next_page = driver.find_element(By.CLASS_NAME, 'ListingItemTitle__link')
            page_links = driver.find_elements(By.CLASS_NAME, 'ListingItemTitle__link')

            for item in page_links:
                result_list.append(item.get_attribute('href'))
            print(f'Parsing {page_num} page for {model} in {region}', end='\r')
            page_num += 1
        except NoSuchElementException:
            cnt = False
            print()
            print(f'There are {page_num} pages by {model}')
    
    return result_list

In [610]:
def get_car_info(url):
        
    driver.get(url)
    
    year = int(driver.find_element(By.CLASS_NAME, 'CardInfoRow_year').text.split('\n')[1])
    
    car = driver.find_elements(By.CLASS_NAME,'CardBreadcrumbs__itemText')
    brand = car[3].text
    model = car[4].text
    
    car_url = url
    
    bodytype = driver.find_element(By.CLASS_NAME, 'CardInfoRow_bodytype').text.split('\n')[1]
    kmage = int(re.sub('\D', '', driver.find_element(By.CLASS_NAME, 'CardInfoRow_kmAge').text))
    color = driver.find_element(By.CLASS_NAME, 'CardInfoRow_color').text.split('\n')[1]
    
    engine = driver.find_element(By.CLASS_NAME, 'CardInfoRow_engine').text.split('/')
    engineDisplacement = float(re.findall('(\d+.\d+)', engine[0])[0])
    enginePower = int(re.findall('\d+', engine[1])[0])
    fuelType = engine[2]
    
    super_gen = json.loads(driver.find_element(By.ID, 'sale-data-attributes').get_attribute('data-bem'))['sale-data-attributes']
    
    vehicleTransmission = driver.find_element(By.CLASS_NAME, 'CardInfoRow_transmission').text.split('\n')[1]
    drive = driver.find_element(By.CLASS_NAME, 'CardInfoRow_drive').text.split('\n')[1]
    wheel = driver.find_element(By.CLASS_NAME, 'CardInfoRow_wheel').text.split('\n')[1]
    state = driver.find_element(By.CLASS_NAME, 'CardInfoRow_state').text.split('\n')[1]
    owner = driver.find_element(By.CLASS_NAME, 'CardInfoRow_ownersCount').text.split('\n')[1]
    pts = driver.find_element(By.CLASS_NAME, 'CardInfoRow_pts').text.split('\n')[1]
    customs = driver.find_element(By.CLASS_NAME, 'CardInfoRow_customs').text.split('\n')[1]
    
    try:
        owningTime = driver.find_element(By.CLASS_NAME, 'CardInfoRow_owningTime').text.split('\n')[1]
    except NoSuchElementException:
        owningTime = None
    
    try:
        description = driver.find_element(By.CLASS_NAME, 'CardDescriptionHTML').text
    except NoSuchElementException:
        description = None
    
    sell_id = int(re.findall('\d+', driver.find_element(By.CLASS_NAME, 'CardHead__id').text)[0])
    price = int(re.sub('\D', '', driver.find_element(By.CLASS_NAME, 'OfferPriceCaption__price').text))
    
    complect_list = []
    
    complect_data = driver.find_elements(By.CLASS_NAME, 'ComplectationGroupsDesktop__itemList')

    for item in complect_data:
        complect_list.extend(item.text.replace('\n', '').split('•')[1:])
    
    catalog_url = driver.find_element(By.CLASS_NAME, 'CardCatalogLink').get_attribute('href')
    
    data_dict = {
        'brand': brand, 'model': model, 'year': year, 'bodytype': bodytype, 'kmage': kmage,
        'color': color, 'engineDisplacement': engineDisplacement, 'enginePower': enginePower, 
        'fuelType': fuelType, 'super_gen': super_gen, 'vehicleTransmission': vehicleTransmission,
        'drive': drive, 'wheel': wheel, 'state': state, 'owner': owner, 'pts': pts, 'customs': customs, 
        'owningTime': owningTime, 'description': description, 'sell_id': sell_id, 'price': price, 
        'car_url': car_url, 'catalog_url': catalog_url, 'equipment_dict': complect_list
    }
    
    return data_dict

In [661]:
def get_dicts_from_catalog(url):
    
    driver.get(url)

    car_info_full = driver.find_elements(By.CLASS_NAME, 'list-values')
    
    car_info_dict_ru = {}
    
    for item in car_info_full:
        el = item.text.split('\n')
        for i in range(0, len(el) - 1, 2):
            car_info_dict_ru[el[i]] = el[i + 1]
    
    engine_type_dict = {'бензин': 'GASOLINE', 'дизель': 'DIESEL', 
                        'гибрид': 'HYBRID', 'электро': 'ELECTRO', 'газ': 'LPG'}
    gear_type_dict = {'передний': 'FORWARD_CONTROL', 'полный': 'ALL_WHEEL_DRIVE', 'задний': 'REAR_DRIVE'}
    transmission_dict = {'автомат': 'AUTOMATIC', 'робот': 'ROBOT', 
                         'механика': 'MECHANICAL', 'вариатор': 'VARIATOR'}

    car_info_dict_en = {
        'engine_type': engine_type_dict[car_info_dict_ru['Тип двигателя']],
        'gear_type': gear_type_dict[car_info_dict_ru['Привод']],
        'transmission': transmission_dict[car_info_dict_ru['Коробка']],
        'power': int(re.findall('\d+', car_info_dict_ru['Мощность'])[0]),
        'power_kvt': int(re.findall('\d+', car_info_dict_ru['Максимальная мощность, л.с./кВт при об/мин'])[1]),
        'acceleration': car_info_dict_ru.get('Разгон до 100 км/ч, с'),
        'clearance_min': min(map(int, re.findall('\d+', car_info_dict_ru['Клиренс']))),
        'fuel_rate': float(car_info_dict_ru['Расход топлива, л город/трасса/смешанный'].split('/')[2])
    }
    
    url2 = url.replace('specifications', 'equipment')
    
    driver.get(url2)

    equipment_list = []

    equipment_data = driver.find_elements(By.CLASS_NAME, 'catalog__package-list-i')
    for item in equipment_data:
        equipment_list.append(item.text)
    
    car_data = json.loads(driver.find_element(By.CLASS_NAME, 'search-form-v2-mmm').get_attribute('data-bem'))
    equip_data = json.loads(driver.find_element(By.CLASS_NAME, 'catalog__section').get_attribute('data-bem'))
    attrib_data = json.loads(driver.find_element(By.CLASS_NAME, 'sale-data-attributes').get_attribute('data-bem'))
    
    return {'super_gen_2': car_info_dict_en, 'super_gen_3': attrib_data, 'complectation_dict': equipment_list}

In [538]:
# for key, item in equip_data['catalog-equipment-controller']['options'].items():
#     print(item.get('options'))

## Gathering URLs from MODEL pages in Regions

In [607]:
start_time = time.time()
model_spb_urls = get_urls_for_model_in_region('infiniti', 'sankt-peterburg')
print(time.time() - start_time)

Parsing 3 page for infiniti in sankt-peterburg
There are 4 pages by infiniti
17.838968753814697


## Getting main info from car page

In [612]:
for url in tqdm(model_spb_urls):
    try:
        valid_data = valid_data.append(get_car_info(url), ignore_index=True)
    except NoSuchElementException:
        print('Error 404', end='\r')

 16%|█████████████▏                                                                   | 17/104 [00:48<03:20,  2.31s/it]

100%|████████████████████████████████████████████████████████████████████████████████| 104/104 [05:37<00:00,  3.25s/it]


In [616]:
valid_data.sample(3).T

,182,170,138
year,2014,2007,2011
bodytype,внедорожник 5 дв.,внедорожник 5 дв.,внедорожник 5 дв.
kmage,95784,223000,160000
color,жёлтый,оранжевый,чёрный
engineDisplacement,3.7,4.5,3.7
enginePower,333,328,333
fuelType,Бензин,Бензин,Бензин
vehicleTransmission,автоматическая,автоматическая,автоматическая
drive,полный,полный,полный
wheel,Левый,Левый,Левый


## Getting additional information from catalog (links from main DF)

In [662]:
for car_url in tqdm(valid_data['catalog_url'].value_counts().index.tolist()):
    try:
        catalog_data = catalog_data.append(get_dicts_from_catalog(car_url), ignore_index=True)
    except NoSuchElementException:
        print('Error 404', end='\r')

100%|██████████████████████████████████████████████████████████████████████████████████| 48/48 [04:22<00:00,  5.47s/it]


In [663]:
catalog_data

,complectation_dict,equipment_dict,model_info,super_gen_2,super_gen_3
0,NaN,NaN,NaN,"{'engine_type': 'GASOLINE', 'gear_type': 'ALL_...","{'sale-data-attributes': {'asciiCat': 'cars', ..."
1,NaN,NaN,NaN,"{'engine_type': 'GASOLINE', 'gear_type': 'ALL_...","{'sale-data-attributes': {'asciiCat': 'cars', ..."
2,NaN,NaN,NaN,"{'engine_type': 'GASOLINE', 'gear_type': 'ALL_...","{'sale-data-attributes': {'asciiCat': 'cars', ..."
3,NaN,NaN,NaN,"{'engine_type': 'GASOLINE', 'gear_type': 'ALL_...","{'sale-data-attributes': {'asciiCat': 'cars', ..."
4,NaN,NaN,NaN,"{'engine_type': 'GASOLINE', 'gear_type': 'REAR...","{'sale-data-attributes': {'asciiCat': 'cars', ..."
...,...,...,...,...,...
162,[],NaN,NaN,"{'engine_type': 'GASOLINE', 'gear_type': 'ALL_...","{'sale-data-attributes': {'asciiCat': 'cars', ..."
163,"[Спортивные передние сиденья, Кожа (Материал с...",NaN,NaN,"{'engine_type': 'DIESEL', 'gear_type': 'ALL_WH...","{'sale-data-attributes': {'asciiCat': 'cars', ..."
164,"[Круиз-контроль, Электропривод зеркал, Бортово...",NaN,NaN,"{'engine_type': 'GASOLINE', 'gear_type': 'ALL_...","{'sale-data-attributes': {'asciiCat': 'cars', ..."
165,"[Круиз-контроль, Мультифункциональное рулевое ...",NaN,NaN,"{'engine_type': 'GASOLINE', 'gear_type': 'REAR...","{'sale-data-attributes': {'asciiCat': 'cars', ..."
